In [475]:
import pandas as pd
import numpy as np

In [476]:
survey_raw = pd.read_excel('survey.xlsx', index_col=0) 
travel_raw = pd.read_excel('travel.xlsx') 

/Users/georgievakristina/.pyenv/versions/3.11.2/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [477]:
survey_raw.columns = ['name', 'lob', 'role', 'location', 'attending', '9 (Q9) ',
       '12 (Q12_1) ', '12 (Q12_2) ', '12 (Q12_3) ', '12 (Q12_4) ',
       '12 (Q12_5) ', '12 (Q12_6) ', '12 (Q12_7) ', '12 (Q12SPECIFIED_6) ',
       'travel_notes', 'doing-talk', 'talk-title', 'talk-description', '18 (Q18) ',
       'lob_rep_interest', '20 (Q20) ', '21 (Q21) ', '23 (Q23) ', 'lob_rep_reason',
       '25 (Q25_1) ', '26 (Q26_1) ', 'topics', 'other', 'Started',
       'Completed', 'Branched Out', 'Over Quota', 'Last Modified']

In [478]:
travel_raw["is_host"] = travel_raw['Location'] == travel_raw['Travel to']
travel = travel_raw[['Location', 'Travel to', 'People', 'Est total travel pp', 'is_host']].dropna().copy()
travel['People'] = travel['People'].astype(int)
travel.columns = ['location', 'destination', 'total_people', 'estimated_travel', 'is_host']
travel['location'] = travel['location'].str.lower()
travel['destination'] = travel['destination'].str.lower()
travel['location'] = travel['location'].str.replace('washington, d.c.', 'washington dc')
travel['destination'] = travel['destination'].str.replace('washington, d.c.', 'washington dc')
travel['destination'] = travel['destination'].str.replace('washington', 'washington dc')
travel['destination'] = travel['destination'].str.replace('washington dc dc', 'washington dc')
travel['destination'] = travel['destination'].str.replace('manhattan beach', 'manhattan beach & la')
travel['location'] = travel['location'].str.replace('manhattan beach', 'manhattan beach & la')
travel['destination'] = travel['destination'].str.replace('ã', 'a')
travel['location'] = travel['location'].str.replace('ã', 'a')

/var/folders/4w/v5gq_r_d3x5bfsq8nd08gj0c0000gn/T/ipykernel_36179/3116241812.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  travel['location'] = travel['location'].str.replace('washington, d.c.', 'washington dc')
/var/folders/4w/v5gq_r_d3x5bfsq8nd08gj0c0000gn/T/ipykernel_36179/3116241812.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  travel['destination'] = travel['destination'].str.replace('washington, d.c.', 'washington dc')


In [479]:
travel['destination'].unique()

array(['bengaluru', 'paris', 'dallas', 'barcelona', 'shanghai',
       'sao paulo', 'berlin', 'milan', 'munich', 'frankfurt', 'stockholm',
       'bogota', 'boston', 'washington dc', 'new york', 'london',
       'casablanca', 'chicago', 'seattle', 'gurugram', 'zurich', 'madrid',
       'manhattan beach & la', 'santiago', 'san francisco', 'sydney',
       'toronto', 'perth', 'tokyo', 'singapore'], dtype=object)

In [480]:
survey_raw = survey_raw[survey_raw['location'].str.len() >1]
survey_raw = survey_raw[survey_raw['attending'] == 'Yes']
survey_raw = survey_raw[survey_raw['location'] != 'dfasd']

In [481]:
#clean locations
survey_raw['location'] = survey_raw['location'].str.lower()
survey_raw['location'] = survey_raw['location'].str.replace('chk - ', '')
survey_raw['location'] = survey_raw['location'].str.replace('acc', '')
survey_raw['location'] = survey_raw['location'].str.replace('dal', 'dallas')
survey_raw['location'] = survey_raw['location'].str.replace('guk', 'gurugram')
survey_raw['location'] = survey_raw['location'].str.replace('bos', 'boston')
survey_raw['location'] = survey_raw['location'].str.replace('bostonton', 'boston')
survey_raw['location'] = survey_raw['location'].str.replace('cas', 'casablanca')
survey_raw['location'] = survey_raw['location'].str.replace('casablancaablanca', 'casablanca')
survey_raw['location'] = survey_raw['location'].str.replace(',', '')
survey_raw['location'] = survey_raw['location'].str.replace('-', '')
survey_raw['location'] = survey_raw['location'].str.replace('dallaslaslas', 'dallas')
survey_raw['location'] = survey_raw['location'].str.replace('dallaslas', 'dallas')
survey_raw['location'] = survey_raw['location'].str.replace('á', 'a')
survey_raw['location'] = survey_raw['location'].str.replace('ã', 'a')
survey_raw['location'] = survey_raw['location'].str.replace('ü', 'u')
survey_raw['location'] = survey_raw['location'].str.replace(' india', '')
survey_raw['location'] = survey_raw['location'].str.replace(' south africa', '')
survey_raw['location'] = survey_raw['location'].str.replace(' morocco', '')
survey_raw['location'] = survey_raw['location'].str.replace('gurgaon', 'gurugram')
survey_raw['location'] = survey_raw['location'].str.replace('gurugaon', 'gurugram')
survey_raw['location'] = survey_raw['location'].str.replace('bangalore', 'bengaluru')
survey_raw['location'] = survey_raw['location'].str.replace('  bek', '')
survey_raw['location'] = survey_raw['location'].str.replace('bek', 'bengaluru')
survey_raw['location'] = survey_raw['location'].str.replace('santaigo', 'santiago')
survey_raw['location'] = survey_raw['location'].str.replace(' brazil', '')
survey_raw['location'] = survey_raw['location'].str.replace(' city', '')
survey_raw['location'] = survey_raw['location'].str.replace('sfo', 'san francisco')
survey_raw['location'] = survey_raw['location'].str.replace('was', 'washington dc')
survey_raw['location'] = survey_raw['location'].str.replace('dchington dc', 'dc')
survey_raw['location'] = survey_raw['location'].str.replace('saettle', 'seattle')
survey_raw['location'] = survey_raw['location'].str.replace('colgone', 'cologne')
survey_raw['location'] = survey_raw['location'].str.replace('los angeles', 'temp')
survey_raw['location'] = survey_raw['location'].str.replace('manhattan beach', 'temp')
survey_raw['location'] = survey_raw['location'].str.replace('nyc', 'new york')
survey_raw['location'] = survey_raw['location'].str.replace('sdasd', 'madrid')
survey_raw['location'] = survey_raw['location'].str.replace('temp', 'manhattan beach & la')
survey_raw['location'] = survey_raw['location'].str.strip()

In [482]:
unique_locations = survey_raw['location'].unique()
unique_locations.sort()
unique_locations

array(['amsterdam', 'atlanta', 'austin', 'barcelona', 'bengaluru',
       'berlin', 'bogota', 'boston', 'brooklyn', 'brussels', 'casablanca',
       'chicago', 'cologne', 'copenhagen', 'dallas', 'delhi', 'denver',
       'dusseldorf', 'frankfurt', 'gurugram', 'helsinki', 'houston',
       'india', 'johannesburg', 'london', 'madrid',
       'manhattan beach & la', 'melbourne', 'miami', 'milan', 'mumbai',
       'munich', 'new delhi', 'new jersey', 'new york', 'oslo', 'paris',
       'perth', 'pittsburgh', 'rio de janeiro', 'san deigo',
       'san francisco', 'santiago', 'sao paulo', 'seattle', 'seoul',
       'shanghai', 'shenzhen', 'silicon valley', 'singapore', 'stockholm',
       'sydney', 'tokyo', 'toronto', 'vienna', 'warsaw', 'washington dc',
       'zurich'], dtype=object)

# General counts

In [483]:
survey_raw['doing-talk-clean'] =  survey_raw['doing-talk'] ==  'Yes'
survey_raw['doing-talk-clean'] = np.where(survey_raw['doing-talk-clean'] == False, np.nan, survey_raw['doing-talk-clean'])
#survey_raw[['doing-talk-clean', 'doing-talk'] ] 
office_counts = survey_raw.groupby('location').count()[['name', 'doing-talk-clean']]

In [484]:
combined = office_counts.join(travel.set_index('location'), how='left')
combined.columns = ['total_registered', 'total_doing_talk', 'destination', 'total_people', 'estimated_travel', 'is_host']

In [485]:
combined[combined['is_host'] == True].reset_index()

,location,total_registered,total_doing_talk,destination,total_people,estimated_travel,is_host
0,barcelona,8,0,barcelona,27.0,0,True
1,bengaluru,24,2,bengaluru,140.0,0,True
2,berlin,25,7,berlin,51.0,0,True
3,bogota,10,0,bogota,20.0,0,True
4,boston,34,3,boston,97.0,0,True
5,casablanca,13,6,casablanca,43.0,0,True
6,chicago,33,2,chicago,68.0,0,True
7,dallas,4,0,dallas,13.0,0,True
8,frankfurt,5,1,frankfurt,19.0,0,True
9,gurugram,36,5,gurugram,169.0,0,True


In [486]:
combined.to_csv("data/extracted_registrations.csv")

# Travellers list

In [487]:
travel_subset = travel[travel['total_people'] > 1]
merged_with_travel = pd.merge(survey_raw,travel_subset,how='left',left_on='location',right_on='location')
small_office_travellers = merged_with_travel[merged_with_travel['is_host'] == False].copy()

In [488]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

small_office_travellers['estimated_travel_plus_perc'] = small_office_travellers['estimated_travel'] * 1.30
small_office_travellers[['name', 'role', 'lob', 'location', 'destination', 'estimated_travel_plus_perc', 'doing-talk-clean', 'travel_notes', 'other']]

,name,role,lob,location,destination,estimated_travel_plus_perc,doing-talk-clean,travel_notes,other
0,Eivind Strøm,Data Scientist,X AI & Software,oslo,stockholm,364.0,NaN,NaN,.
1,Valentina Jarovaja,Data & Analytics Lead,X Delivery,dusseldorf,frankfurt,546.0,NaN,NaN,NaN
3,Marion Nöske,Senior Director Data & Analytics - X Delivery,X Delivery,dusseldorf,frankfurt,546.0,NaN,NaN,thank you for organizing!
5,Michael Logen,Data & Analytics Lead,X Delivery,dusseldorf,frankfurt,546.0,NaN,NaN,-
19,Martin Schleicher,Data & Analytics Lead – X Delivery,X Delivery,dusseldorf,frankfurt,546.0,NaN,NaN,Don't make optional questions mandatory.
24,Otto Saikkonen,Senior Data Scientist,X AI & Software,helsinki,stockholm,403.0,NaN,NaN,No
31,Dante Cicco,Senior Engineer,X AI & Software,pittsburgh,washington dc,911.3,NaN,No,Nope!
35,Jo Saakvitne,Lead Data Scientist,X AI & Software,oslo,stockholm,364.0,NaN,No,No
39,Petter Moen,Senior Data Scientist,X AI & Software,oslo,stockholm,364.0,NaN,NaN,NaN
42,Valentina Jarovaja,Data & Analytics Lead,X Delivery,dusseldorf,frankfurt,546.0,NaN,NaN,I would like to have enough time to collaborat...


# Host office representation

In [489]:
lob_counts = merged_with_travel.groupby(['destination', 'lob']).count()['name'].reset_index()
lob_counts_pivoted = pd.pivot(lob_counts, columns=['lob'], index=['destination'])['name'][['X AI & Software', 'X Delivery', 'X Ventures']]

In [490]:
lob_counts_pivoted

lob,X AI & Software,X Delivery,X Ventures
destination,,,
barcelona,4.0,4.0,NaN
bengaluru,3.0,23.0,NaN
berlin,11.0,2.0,20.0
bogota,1.0,9.0,NaN
boston,17.0,17.0,NaN
casablanca,1.0,11.0,1.0
chicago,10.0,22.0,1.0
dallas,14.0,1.0,1.0
frankfurt,2.0,11.0,NaN


In [491]:
lob_rep_plan = pd.read_excel('lob_rep_plan.xlsx') 
lob_rep_plan['Missing LOB'] = lob_rep_plan['Missing LOB'].str.split(',')
lob_rep_plan = lob_rep_plan.explode(['Missing LOB'])

# LOB reps

In [492]:
lob_reps = survey_raw[(~survey_raw['lob_rep_interest'].isna()) & (survey_raw['lob_rep_interest'] != 'None of the above')]


In [493]:
lob_reps.to_csv("data/extracted_lob_reps.csv")

### AI & software

In [494]:
lob_counts_pivoted[lob_counts_pivoted['X AI & Software'].isna()]

lob,X AI & Software,X Delivery,X Ventures
destination,,,


In [495]:
lob_reps[lob_reps['lob'] == 'X AI & Software'].dropna(subset=['lob_rep_reason'])

,name,lob,role,location,attending,9 (Q9),12 (Q12_1),12 (Q12_2),12 (Q12_3),12 (Q12_4),12 (Q12_5),12 (Q12_6),12 (Q12_7),12 (Q12SPECIFIED_6),travel_notes,doing-talk,talk-title,talk-description,18 (Q18),lob_rep_interest,20 (Q20),21 (Q21),23 (Q23),lob_rep_reason,25 (Q25_1),26 (Q26_1),topics,other,Started,Completed,Branched Out,Over Quota,Last Modified,doing-talk-clean
Record ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
638295193090591825,Guillem Borrell Nogueras,X AI & Software,Lead AI Engineer,madrid,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,Yes,All you need is Duckdb,Duckdb is a relatively new high-performance qu...,45 minutes,I am interested in being LOB representative in...,NaN,No,NaN,I've been teaching technical stuff to DS and S...,NaN,NaN,Anything that is interesting and not about GenAI,"Please, don't make it only about GenAI",2023-09-05 14:01:49,2023-09-05 14:13:08,NaN,NaN,2023-09-05 14:13:08,1.0
638298051723847524,Archit Matta,X AI & Software,Data Scientist,new york,Yes,No,-,-,-,-,-,"Others ( please specify, including allergies)",-,No Beef,NaN,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,Yes,NaN,I'm super excited and have tried to contribute...,NaN,NaN,I'd be interested in knowing about more networ...,I would also be super interested in helping pl...,2023-09-08 21:26:12,2023-09-08 21:34:28,NaN,NaN,2023-09-08 21:34:28,NaN
638301162078151552,Abby Gong,X AI & Software,Senior Data Scientist,boston,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,Yes,MedTech supply chain AI capabilities,The application of supply chain AI capabilitie...,15 minutes,I am interested in being LOB representative in...,NaN,No,NaN,"I’m from AI&Software, but I have done multiple...",NaN,NaN,"Future one BCGx offering, collaborative ways o...",would like to hear the envision from org team ...,2023-09-12 11:50:07,2023-09-12 12:01:12,NaN,NaN,2023-09-12 19:37:38,1.0
638301202864066433,Kamal Darchini,X AI & Software,Lead data scientist,toronto,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,No,NaN,I have been in this LOB for 3+ years and have ...,NaN,NaN,Hands on GenAI,Thanks for organizing!,2023-09-12 12:58:06,2023-09-12 13:11:28,NaN,NaN,2023-09-12 13:11:28,NaN
638301255487576553,Saurabh Mahajan,X AI & Software,AI Engineer,denver,Yes,Yes,-,Vegetarian,-,-,-,-,-,NaN,NaN,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,No,NaN,Denver office has a very small presence of gam...,NaN,NaN,-,Can you send a broad categories and informatio...,2023-09-12 14:25:48,2023-09-12 14:40:44,NaN,NaN,2023-09-12 14:40:44,NaN
638312276035642564,Olga Kotova,X AI & Software,Data Scientist,london,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,Yes,Transforming Drug Discovery workflow using AI/...,Global Biopharma company was looking to accele...,30 minutes,I am interested in being LOB representative in...,NaN,No,NaN,I have strong public speaking skills emerging ...,NaN,NaN,Generative algorithms and language models,-,2023-09-25 08:33:23,2023-09-25 08:45:57,NaN,NaN,2023-09-25 08:45:57,1.0
638315193987323543,Kaoru Schwarzenegger,X AI & Software,Data Scientist,tokyo,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,No,NaN,Since I have a wide field of interest apart fr...,NaN,NaN,Different perspectives from other LOBs and ide...,Will likely not be able to block the whole day...,2023-09-28 17:36:38,2023-09-28 17:43:25,NaN,NaN,2023-09-28 17:43:25,NaN
638315202876045630,Adam Benali,X AI & Software,AI Engineer,stockholm,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,No,NaN,"We pretty much only have one LOB in Stockholm,...",NaN,NaN,-,-,2023-09-28 17:51:27,2023-09-28 17:57:33,NaN,NaN,2023-09-28 17:57:33,NaN
638315430336436294,Dave Rattan,X AI & Software,AI Engineer,sydney,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,Yes,How to mo

### X Delivery

In [496]:
lob_counts_pivoted[lob_counts_pivoted['X Delivery'].isna()]

lob,X AI & Software,X Delivery,X Ventures
destination,,,
milan,15.0,NaN,NaN
paris,33.0,NaN,NaN
santiago,11.0,NaN,NaN
singapore,14.0,NaN,10.0
sydney,27.0,NaN,3.0
tokyo,7.0,NaN,4.0
washington dc,10.0,NaN,NaN
zurich,7.0,NaN,NaN


In [497]:
len(lob_counts_pivoted[lob_counts_pivoted['X Delivery'].isna()].index)

8

In [498]:
lob_reps[lob_reps['lob'] == 'X Delivery'].dropna(subset=['lob_rep_reason'])

,name,lob,role,location,attending,9 (Q9),12 (Q12_1),12 (Q12_2),12 (Q12_3),12 (Q12_4),12 (Q12_5),12 (Q12_6),12 (Q12_7),12 (Q12SPECIFIED_6),travel_notes,doing-talk,talk-title,talk-description,18 (Q18),lob_rep_interest,20 (Q20),21 (Q21),23 (Q23),lob_rep_reason,25 (Q25_1),26 (Q26_1),topics,other,Started,Completed,Branched Out,Over Quota,Last Modified,doing-talk-clean
Record ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
638298890718873054,Amine Benaddi,X Delivery,Engineering Lead,casablanca,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,Yes,Leveraging typescript for micro service,"Trpc, turbo, nodejs",30 minutes,I am interested in being LOB representative in...,NaN,No,NaN,Interesting topic for next projects,NaN,NaN,Architectures leveraged in other cases,"Comparing Astro, Nextjs, fresh, blitzjs, etc",2023-09-09 20:44:31,2023-09-09 20:49:39,NaN,NaN,2023-09-09 20:49:39,1.0
638307984685797820,Panagiota Papakosta,X Delivery,Senior Analyst,munich,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,Yes,GeoAnalytics to accelerate Climate & Sustainab...,what is understood by the science of where and...,15 minutes,I am interested in being LOB representative in...,NaN,Yes,NaN,happy to represent LOB in the Munich office,NaN,NaN,other C&S applications,thanks for orga,2023-09-20 09:21:08,2023-09-20 09:27:17,NaN,NaN,2023-09-20 09:27:17,1.0
638309142512518582,Purnima Aggarwal,X Delivery,Lead data analyst,new delhi,Yes,Yes,Vegan,Vegetarian,-,-,-,"Others ( please specify, including allergies)",-,No onion and no garlic,No,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,Yes,NaN,Not sure,NaN,NaN,Outlook,No,2023-09-21 17:30:51,2023-09-21 17:33:52,NaN,NaN,2023-09-21 17:34:01,NaN
638309193066578398,Amar Patel,X Delivery,Product Management Director,austin,Yes,Yes,-,-,-,-,-,-,No dietary restrictions,NaN,No,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,Yes,NaN,I interface across all the areas within the X ...,NaN,NaN,GenAI use cases that we have developed and dep...,no,2023-09-21 18:55:06,2023-09-21 19:00:59,NaN,NaN,2023-09-21 19:00:59,NaN
638309418719094351,Jason He,X Delivery,Senior Delivery Analyst,shenzhen,Yes,Yes,-,-,-,-,-,-,No dietary restrictions,NaN,No,Yes,"OrgBuilder, data-driven organizational redesig...",Showcase and demo on OrgBuilder tool,30 minutes,I am interested in being LOB representative in...,NaN,No,NaN,There are only 2 X Delivery people in Greater ...,NaN,NaN,Sharing (case / tool) from Venture team,No,2023-09-22 01:11:11,2023-09-22 01:20:59,NaN,NaN,2023-09-22 01:20:59,1.0
638312379032342134,Oliver Hof,X Delivery,Technical Delivery Sr. Manager,warsaw,Yes,Yes,-,-,-,-,-,-,-,NaN,No,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,Yes,NaN,Almost 8 years in BCG; have been in the same j...,NaN,NaN,How teams are using AI tools internally. Any r...,"I am not quite sure how the LOB Rep will work,...",2023-09-25 11:25:03,2023-09-25 11:30:19,NaN,NaN,2023-09-25 11:30:19,NaN
638314001137590077,mohd sharique ansari,X Delivery,engineering lead,gurugram,Yes,No,Vegan,Vegetarian,-,-,-,-,-,NaN,NaN,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,Yes,NaN,I am part of the most revenue generating produ...,NaN,NaN,"Software Security, Infra modernization techniques",no,2023-09-27 08:28:33,2023-09-27 08:37:48,NaN,NaN,2023-09-27 08:37:48,NaN
638314079246651573,Madhur Mukhi,X Delivery,Engineering Lead,gurugram,Yes,No,-,Vegetarian,-,-,-,-,-,NaN,NaN,Yes,How we brought a legacy product in a Multi ten...,Workspace Manager is a utility to host and man...,15 minutes,I am interested in being LOB representative in...,NaN,Yes,NaN,Yes,NaN,NaN,"Agile , Software development",NaN,2023-09-27 10:38:44,2023-09-27 10:52:54,NaN,NaN,2023-09-27 10:52:54,1.0
638315013930037917,Luísa Lichfett,X Delivery,Software Engineer,sao paulo,Yes,Yes,-,-,-,-,-,-,No dietary restrictions,NaN,Not that I know of,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,No,NaN,I believe that a LOB should have some pa

# Options for LOB Rep X.Delivery

- Amine Benaddi -> Paris or Zurich
- Panagiota Papakosta -> Paris or Zurich or Milan
- Oliver Hof -> Zurich
- David Hoyos -> Santiago
- Deepak Trehan -> Washington DC
- Jason He -> Tokyo
- Sukrit Kurle -> Singapore

In [499]:
travel_cmn_cdg = 330
travel_cmn_zrh = 600 #takes too long vs cdg


travel_muc_cdg = 260
travel_muc_zrh = 450
travel_muc_mxp = 400

travel_waw_zrh = 250

hotel_mxp = 200
hotel_zrh = 200
hotel_cdg = 300


travel_bog_scl = 11000 #7hr flight, lol
hotel_scl = 400 #2 nights required

travel_bos_iad = 200
hotel_iad = 400

travel_szx_tok = 2100
hotel_tok = 300

travel_blr_sin = 2100 #5hr flight
hotel_sin = 600 #2 nights required

total_cmn_cdg = travel_cmn_cdg + hotel_cdg
total_cmn_zrh = travel_cmn_zrh + hotel_zrh

total_muc_cdg = travel_muc_cdg + hotel_cdg
total_muc_zrh = travel_muc_zrh + hotel_zrh
total_muc_mxp = travel_muc_mxp + hotel_mxp

total_scl = travel_bog_scl + hotel_scl
total_iad = travel_bos_iad + hotel_iad
total_tok = travel_szx_tok + hotel_tok
total_sin = travel_blr_sin + hotel_sin
total_waw_zrh = travel_waw_zrh + hotel_zrh


travel_lob_rep_delivery = lob_reps[lob_reps['lob'] == 'X Delivery'].dropna(subset=['lob_rep_reason'])
travel_lob_rep_delivery = travel_lob_rep_delivery.loc[travel_lob_rep_delivery['name'].isin(['Amine Benaddi', 'Panagiota Papakosta', 'Oliver Hof', 'David Hoyos', 'Deepak Trehan', 'Jason He', 'Sukrit Kurle'])]
travel_lob_rep_delivery = travel_lob_rep_delivery.drop_duplicates(subset='name')

travel_lob_rep_delivery.insert(4, 'destination', ['paris', 'milan', 'tokyo', 'zurich', 'singapore', 'washington dc', 'santiago'], True)
travel_lob_rep_delivery.insert(5, 'estimated_travel_plus_perc', [total_cmn_cdg, total_muc_mxp, total_tok, total_waw_zrh, total_sin, total_iad, total_scl], True)


travel_lob_rep_delivery[['name', 'lob', 'role', 'location', 'destination', 'estimated_travel_plus_perc', 'doing-talk-clean', 'travel_notes', 'other', 'lob_rep_reason']]



,name,lob,role,location,destination,estimated_travel_plus_perc,doing-talk-clean,travel_notes,other,lob_rep_reason
Record ID,,,,,,,,,,
638298890718873054,Amine Benaddi,X Delivery,Engineering Lead,casablanca,paris,630,1.0,NaN,"Comparing Astro, Nextjs, fresh, blitzjs, etc",Interesting topic for next projects
638307984685797820,Panagiota Papakosta,X Delivery,Senior Analyst,munich,milan,600,1.0,NaN,thanks for orga,happy to represent LOB in the Munich office
638309418719094351,Jason He,X Delivery,Senior Delivery Analyst,shenzhen,tokyo,2400,1.0,No,No,There are only 2 X Delivery people in Greater ...
638312379032342134,Oliver Hof,X Delivery,Technical Delivery Sr. Manager,warsaw,zurich,450,NaN,No,"I am not quite sure how the LOB Rep will work,...",Almost 8 years in BCG; have been in the same j...
638315943002284580,Sukrit Kurle,X Delivery,Data Engineering Lead,bengaluru,singapore,2700,NaN,NaN,I will collaborate with Joerg Schneider to pre...,I have been a part of the X Delivery Data Engi...
638315943121595054,Deepak Trehan,X Delivery,Data & Analytics Director,boston,washington dc,600,1.0,NaN,Majid Alivand,Yes
638316005914858606,David Hoyos,X Delivery,Data Analytics Lead,bogota,santiago,11400,NaN,NaN,"what are the available topics for talks, becau...",X Delivery Lead that knows how the LOB works a...


# Ventures

In [500]:
lob_counts_pivoted[lob_counts_pivoted['X Ventures'].isna()]

lob,X AI & Software,X Delivery,X Ventures
destination,,,
barcelona,4.0,4.0,NaN
bengaluru,3.0,23.0,NaN
bogota,1.0,9.0,NaN
boston,17.0,17.0,NaN
frankfurt,2.0,11.0,NaN
gurugram,2.0,38.0,NaN
madrid,12.0,15.0,NaN
milan,15.0,NaN,NaN
munich,7.0,4.0,NaN


In [501]:
len(lob_counts_pivoted[lob_counts_pivoted['X Ventures'].isna()].index)

21

In [502]:
lob_reps[lob_reps['lob'] == 'X Ventures'].dropna(subset=['lob_rep_reason'])

,name,lob,role,location,attending,9 (Q9),12 (Q12_1),12 (Q12_2),12 (Q12_3),12 (Q12_4),12 (Q12_5),12 (Q12_6),12 (Q12_7),12 (Q12SPECIFIED_6),travel_notes,doing-talk,talk-title,talk-description,18 (Q18),lob_rep_interest,20 (Q20),21 (Q21),23 (Q23),lob_rep_reason,25 (Q25_1),26 (Q26_1),topics,other,Started,Completed,Branched Out,Over Quota,Last Modified,doing-talk-clean
Record ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
638301182426033496,Robin Weston,X Ventures,Principal,singapore,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,Yes,Building Gen AI PRODUCTS,Who decided that Gen AI doesn't require and de...,15 minutes,I am interested in being LOB representative in...,NaN,No,NaN,I am going to be in London already visiting fa...,NaN,NaN,Would love to hear about some ventures - I hav...,NaN,2023-09-12 12:24:02,2023-09-12 12:28:14,NaN,NaN,2023-09-12 12:28:14,1.0
638315196919892165,Sebastian Geidies,X Ventures,"Partner & Vice President, Engineering",berlin,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,No,NaN,Long experience with X.Ventures,NaN,NaN,Distributed Computing,NaN,2023-09-28 17:41:31,2023-09-28 17:43:46,NaN,NaN,2023-09-28 17:43:46,NaN
638315201269934155,Astera Schneeweisz,X Ventures,VP of Engineering,berlin,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,No,NaN,NaN,NaN,I am interested in being LOB representative in...,NaN,No,NaN,"Because i often work in the role of a GM, and ...",NaN,NaN,"Diversity, cultural topics, eng starter projects",not at this time :),2023-09-28 17:48:46,2023-09-28 17:56:42,NaN,NaN,2023-09-28 17:56:42,NaN
638315258771559218,Constantine Karlis,X Ventures,Lead Engineer,berlin,Yes,No,-,-,-,-,-,-,No dietary restrictions,NaN,NaN,Yes,"Privacy, Enterprise and DLT",How we found the right balance of openness and...,30 minutes,I am interested in being LOB representative in...,NaN,No,NaN,I've been at DV/X Ventures for a very long tim...,NaN,NaN,"In general: cats, noodles, synthesisers. Speci...",For the talk I would like to collaborate with ...,2023-09-28 19:24:37,2023-09-28 19:44:53,NaN,NaN,2023-09-28 19:44:53,1.0


# Options for LOB Rep X.Ventures

- Robin Weston -> Shanghai
- Sebastian Geidies, Astera, Constantine -> Munich, Frankfurt, Milan, Zurich or Stockholm

In [503]:
travel_sin_pvg = 1000 #estimate: can't get through travel portal

hotel_pvg = 260

travel_ber_muc_flight = 330
travel_ber_muc_train = 450 #more like 200 on DB directly

travel_ber_fra_flight = 260 
travel_ber_fra_train = 590 #more like 200 on DB directly

travel_ber_mxp = 300
travel_ber_zrh = 420
travel_ber_arn = 200

hotel_muc = 166
hotel_fra = 130
hotel_mxp = 200
hotel_zrh = 200
hotel_arn = 150

total_pvg = travel_sin_pvg + hotel_pvg
total_muc = travel_ber_muc_flight + hotel_muc
total_fra = travel_ber_fra_flight + hotel_fra
total_mxp = travel_ber_mxp + hotel_mxp
total_zrh = travel_ber_zrh + hotel_zrh
total_arn = travel_ber_arn + hotel_arn

travel_lob_rep_ventures = lob_reps[lob_reps['lob'] == 'X Ventures'].dropna(subset=['lob_rep_reason'])


travel_lob_rep_ventures.insert(4, 'destination', ['shanghai', 'munich', 'frankfurt', 'stockholm'], True)
travel_lob_rep_ventures.insert(5, 'estimated_travel_plus_perc', [total_pvg, total_muc, total_fra, total_arn], True)


travel_lob_rep_ventures[['name', 'lob', 'role', 'location', 'destination', 'estimated_travel_plus_perc', 'doing-talk-clean', 'lob_rep_reason']]



,name,lob,role,location,destination,estimated_travel_plus_perc,doing-talk-clean,lob_rep_reason
Record ID,,,,,,,,
638301182426033496,Robin Weston,X Ventures,Principal,singapore,shanghai,1260,1.0,I am going to be in London already visiting fa...
638315196919892165,Sebastian Geidies,X Ventures,"Partner & Vice President, Engineering",berlin,munich,496,NaN,Long experience with X.Ventures
638315201269934155,Astera Schneeweisz,X Ventures,VP of Engineering,berlin,frankfurt,390,NaN,"Because i often work in the role of a GM, and ..."
638315258771559218,Constantine Karlis,X Ventures,Lead Engineer,berlin,stockholm,350,1.0,I've been at DV/X Ventures for a very long tim...


In [81]:
talks = survey_raw[['name', 'location', 'talk-title', 'talk-description', 'doing-talk-clean', 'doing-talk', '18 (Q18) ']]
talks = talks[talks['doing-talk-clean'] == 1]

# Define a dictionary to map cities to regions
city_to_region = {
    'amsterdam': 'EMESA',
    'atlanta': 'NAMR',
    'austin': 'NAMR',
    'barcelona': 'EMESA',
    'bek': 'EMESA',
    'bengaluru': 'APAC',
    'berlin': 'EMESA',
    'bogota': 'NAMR',
    'boston': 'NAMR',
    'brooklyn': 'NAMR',
    'brussels': 'EMESA',
    'casablanca': 'EMESA',
    'chicago': 'NAMR',
    'cologne': 'EMESA',
    'copenhagen': 'EMESA',
    'dallas': 'NAMR',
    'denver': 'NAMR',
    'dusseldorf': 'EMESA',
    'frankfurt': 'EMESA',
    'gurugram': 'APAC',
    'helsinki': 'EMESA',
    'houston': 'NAMR',
    'india': 'APAC',
    'johannesburg': 'EMESA',
    'london': 'EMESA',
    'madrid': 'EMESA',
    'manhattan beach & la': 'NAMR',
    'melbourne': 'APAC',
    'miami': 'NAMR',
    'milan': 'EMESA',
    'mumbai': 'APAC',
    'munich': 'EMESA',
    'new delhi': 'APAC',
    'new jersey': 'NAMR',
    'new york': 'NAMR',
    'nyc': 'NAMR',
    'oslo': 'EMESA',
    'paris': 'EMESA',
    'perth': 'APAC',
    'pittsburgh': 'NAMR',
    'rio de janeiro': 'NAMR',
    'san francisco': 'NAMR',
    'santiago': 'NAMR',
    'sao paulo': 'NAMR',
    'seattle': 'NAMR',
    'seoul': 'APAC',
    'shanghai': 'APAC',
    'shenzhen': 'APAC',
    'silicon valley': 'NAMR',
    'singapore': 'APAC',
    'stockholm': 'EMESA',
    'sydney': 'APAC',
    'tokyo': 'APAC',
    'toronto': 'NAMR',
    'vienna': 'EMESA',
    'warsaw': 'EMESA',
    'washington dc': 'NAMR',
    'zurich': 'EMESA'
}

time_difference_mapping = {
    'amsterdam': '+02:00:00',
    'atlanta': '-04:00:00',
    'austin': '-05:00:00',
    'barcelona': '+02:00:00',
    'bek': '+02:00:00',
    'bengaluru': '+05:30:00',
    'berlin': '+02:00:00',
    'bogota': '-05:00:00',
    'boston': '-04:00:00',
    'brooklyn': '-04:00:00',
    'brussels': '+02:00:00',
    'casablanca': '+01:00:00',
    'chicago': '-05:00:00',
    'cologne': '+02:00:00',
    'copenhagen': '+02:00:00',
    'dallas': '-05:00:00',
    'denver': '-06:00:00',
    'dusseldorf': '+02:00:00',
    'frankfurt': '+02:00:00',
    'gurugram': '+05:30:00',
    'helsinki': '+03:00:00',
    'houston': '-05:00:00',
    'india': '+05:30:00',
    'johannesburg': '+02:00:00',
    'london': '+01:00:00',
    'madrid': '+02:00:00',
    'manhattan beach & la': '-07:00:00',
    'melbourne': '+10:00:00',
    'miami': '-04:00:00',
    'milan': '+02:00:00',
    'mumbai': '+05:30:00',
    'munich': '+02:00:00',
    'new delhi': '+05:30:00',
    'new jersey': '-04:00:00',
    'new york': '-04:00:00',
    'nyc': '-04:00:00',
    'oslo': '+02:00:00',
    'paris': '+02:00:00',
    'perth': '+08:00:00',
    'pittsburgh': '-04:00:00',
    'rio de janeiro': '-03:00:00',
    'san francisco': '-07:00:00',
    'santiago': '-04:00:00',
    'sao paulo': '-03:00:00',
    'seattle': '-07:00:00',
    'seoul': '+09:00:00',
    'shanghai': '+08:00:00',
    'shenzhen': '+08:00:00',
    'silicon valley': '-07:00:00',
    'singapore': '+08:00:00',
    'stockholm': '+02:00:00',
    'sydney': '+10:00:00',
    'tokyo': '+09:00:00',
    'toronto': '-04:00:00',
    'vienna': '+02:00:00',
    'warsaw': '+02:00:00',
    'washington dc': '-04:00:00',
    'zurich': '+02:00:00'
}

talks['region'] = talks['location'].map(city_to_region)
talks['time-diff'] = talks['location'].map(time_difference_mapping)

talks['time-diff'] = pd.to_timedelta(talks['time-diff'])

talks.sort_values(by=['time-diff'])



ValueError: expected hh:mm:ss format

In [76]:
talks = talks.drop_duplicates(subset='name')

In [77]:
talks.groupby('region')['name'].count().reset_index()

,region,name
0,APAC,20
1,EMESA,24
2,NAMR,21


In [71]:
location_counts = talks.groupby(['region', 'location'])['name'].count().reset_index()

location_counts = location_counts.sort_values(by=['region', 'name'], ascending=[True, False])

location_counts.columns = ['Region', 'Location', 'Talk Count']

location_counts

,Region,Location,Talk Count
1,APAC,gurugram,5
7,APAC,singapore,4
8,APAC,sydney,4
0,APAC,bengaluru,2
3,APAC,perth,2
2,APAC,melbourne,1
4,APAC,seoul,1
5,APAC,shanghai,1
6,APAC,shenzhen,1
9,APAC,tokyo,1


In [75]:
from datetime import datetime, timedelta

# Define the start time for each region
start_times = {
    'APAC': datetime.strptime('09:30:00', '%H:%M:%S'),
    'EMESA': datetime.strptime('09:30:00', '%H:%M:%S'),
    'NAMR': datetime.strptime('10:00:00', '%H:%M:%S'),
}

# Define time slots (20 minutes and 40 minutes)
time_slots = [timedelta(minutes=20), timedelta(minutes=40)]

# Define lunch break time range
lunch_break_start = datetime.strptime('12:00:00', '%H:%M:%S')
lunch_break_end = datetime.strptime('14:00:00', '%H:%M:%S')

# Create a DataFrame with talks sorted by region and talk count
talk_schedule = location_counts.sort_values(by=['Region', 'Talk Count'], ascending=[True, False])

# Initialize the schedule with empty slots
schedule = []

# Loop through the regions and generate the schedule
for region, location, talk_count in talk_schedule.itertuples(index=False):
    start_time = start_times[region]
    time_slot_index = 0  # Initialize with the first time slot (20 minutes)

    # Schedule talks in the region
    for _ in range(talk_count):
        # Check if it's time for a lunch break
        if start_time >= lunch_break_start and start_time <= lunch_break_end:
            # Skip the lunch break time
            start_time = lunch_break_end + timedelta(minutes=20)

        # Calculate the end time for the talk
        end_time = start_time + time_slots[time_slot_index]

        # Append the talk to the schedule
        schedule.append((region, location, start_time.strftime('%H:%M'), end_time.strftime('%H:%M')))

        # Toggle between 20-minute and 40-minute time slots
        time_slot_index = 1 - time_slot_index

        # Update the start time for the next talk
        start_time = end_time

# Convert the schedule to a DataFrame for better visualization
schedule_df = pd.DataFrame(schedule, columns=['Region', 'Location', 'Start Time', 'End Time'])
schedule_df

,Region,Location,Start Time,End Time
0,APAC,gurugram,09:30,09:50
1,APAC,gurugram,09:50,10:30
2,APAC,gurugram,10:30,10:50
3,APAC,gurugram,10:50,11:30
4,APAC,gurugram,11:30,11:50
5,APAC,singapore,09:30,09:50
6,APAC,singapore,09:50,10:30
7,APAC,singapore,10:30,10:50
8,APAC,singapore,10:50,11:30
9,APAC,sydney,09:30,09:50


# Save all travellers

In [505]:
small_office_travellers['reason_for_travel'] = 'small office'
travel_lob_rep_delivery['reason_for_travel'] = 'represent X Delivery in office with gap'
travel_lob_rep_ventures['reason_for_travel'] = 'represent X Ventures in office with gap'

traveller_columns = ['name', 'role', 'lob', 'location', 'destination', 'estimated_travel_plus_perc', 'doing-talk-clean', 'reason_for_travel']
combined_travellers = pd.concat([
    small_office_travellers[traveller_columns],
    travel_lob_rep_delivery[traveller_columns],
    travel_lob_rep_ventures[traveller_columns]
])

In [506]:
columns_for_duplicates = ['name', 'lob', 'location', 'role']
deduplicated_travellers = combined_travellers.drop_duplicates(subset=columns_for_duplicates, keep='last')

deduplicated_travellers.to_csv("data/extracted_travellers.csv")

In [507]:
deduplicated_merged = merged_with_travel.drop_duplicates(subset=columns_for_duplicates)
replaced_travellers = pd.merge(deduplicated_merged, 
                               deduplicated_travellers[columns_for_duplicates + ['destination']], 
                               how='left', on=columns_for_duplicates).reset_index()


In [508]:
deduplicated_travellers

,name,role,lob,location,destination,estimated_travel_plus_perc,doing-talk-clean,reason_for_travel
0,Eivind Strøm,Data Scientist,X AI & Software,oslo,stockholm,364.0,NaN,small office
3,Marion Nöske,Senior Director Data & Analytics - X Delivery,X Delivery,dusseldorf,frankfurt,546.0,NaN,small office
5,Michael Logen,Data & Analytics Lead,X Delivery,dusseldorf,frankfurt,546.0,NaN,small office
19,Martin Schleicher,Data & Analytics Lead – X Delivery,X Delivery,dusseldorf,frankfurt,546.0,NaN,small office
24,Otto Saikkonen,Senior Data Scientist,X AI & Software,helsinki,stockholm,403.0,NaN,small office
31,Dante Cicco,Senior Engineer,X AI & Software,pittsburgh,washington dc,911.3,NaN,small office
35,Jo Saakvitne,Lead Data Scientist,X AI & Software,oslo,stockholm,364.0,NaN,small office
39,Petter Moen,Senior Data Scientist,X AI & Software,oslo,stockholm,364.0,NaN,small office
42,Valentina Jarovaja,Data & Analytics Lead,X Delivery,dusseldorf,frankfurt,546.0,NaN,small office
85,Shobhit Mittal,Data Scientist,X AI & Software,new delhi,gurugram,52.0,NaN,small office


In [509]:
replaced_travellers['destination'] = replaced_travellers['destination_y']
replaced_travellers['destination'] = replaced_travellers['destination'].fillna(replaced_travellers['destination_x'])
replaced_travellers.loc[replaced_travellers["destination"] == 'barcelona', "destination"] = 'madrid'

In [510]:
lob_counts = replaced_travellers.groupby(['destination', 'lob']).count()['name'].reset_index()
lob_counts_pivoted = pd.pivot(lob_counts, columns=['lob'], index=['destination'])['name'][['X AI & Software', 'X Delivery', 'X Ventures']]
lob_counts_pivoted = lob_counts_pivoted.fillna(0)
lob_counts_pivoted['total'] = lob_counts_pivoted['X AI & Software'] + lob_counts_pivoted['X Delivery'] + lob_counts_pivoted['X Ventures']


In [511]:
lob_counts_pivoted

lob,X AI & Software,X Delivery,X Ventures,total
destination,,,,
bengaluru,2.0,18.0,0.0,20.0
berlin,9.0,1.0,15.0,25.0
bogota,1.0,8.0,0.0,9.0
boston,16.0,13.0,0.0,29.0
casablanca,1.0,9.0,1.0,11.0
chicago,9.0,16.0,1.0,26.0
dallas,10.0,1.0,1.0,12.0
frankfurt,2.0,10.0,1.0,13.0
gurugram,2.0,32.0,0.0,34.0
